В задании вам понадобится собрать генеративную модлель для языка

---

# LSTM (7 баллов)

В данной части нужно реализовать модель с ипользованием LSTM

In [1]:
!pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from torch.utils.data import DataLoader
from dataset_maker import DatasetMaker, collate_batch
from data_generator import DataGenerator

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Ниже вам нужно реализовать модель, которая по началу последовательности предсказывает следующий токен.
*   Модель получает на вход последовательность токенов, прогоняет её через LSTM и выдает вероятности следующего токена.  
*   Используйте LSTM из pytorch
*   Не забудьте про `batch_first`

In [4]:
DIRECTORY = './data/books_txt'  
VOCAB_PATH = './data/vocab.txt'
MAX_SEQ_LEN = 256
BATCH_SIZE = 16
OFFSET = MAX_SEQ_LEN // 2
TEST_SIZE = 0.03

In [5]:
dataset_maker = DatasetMaker(directory=DIRECTORY, test_size=TEST_SIZE, vocab_path=VOCAB_PATH, max_seq_len=MAX_SEQ_LEN, batch_size=BATCH_SIZE, offset=OFFSET, create_test_segments=True)
train_dataloader = DataLoader(dataset_maker, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(dataset_maker.segments_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [6]:
dataset_maker.tokenizer.vocab_size

8000

In [7]:
LEARNING_RATE = 0.0005
EMBEDDING_DIM = 512
NUM_LAYERS = 2
PAD_TOKEN_IDX = 0
SAVE_PATH = './saved_models'

In [8]:
model = DataGenerator(vocab_size=dataset_maker.tokenizer.vocab_size, test_dataloader=test_dataloader, embed_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS, pad_token_idx=PAD_TOKEN_IDX, device=device)

### Реализуйте обучение модели:
*  Не забудьте сдвинуть src и trg относительно друг друга.
*  Не забудьте про `clip_grad_norm_`
*  Данных очень много, для отладки лучше проходить только часть данных иначе этоха будет очень длинной

Получите `loss < 5.0` на тестовой выборке. 

Если модель обучается слишком быстро до значений <1.0 вы что-то напутали с данными.

In [ ]:
model.train_loop(epochs=100, train_dataloader=train_dataloader, lr=LEARNING_RATE, save_path=SAVE_PATH)

Epoch 1/100


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 61%|██████    | 10335/17049 [2:20:02<1:29:19,  1.25it/s]

# Генерация текста (5 баллов) 

Реализуйте функцию, которая продолжает текст.
1.   Переведите строчку в токены
2.   Реализуйте код который предсказывает вероятность следующей буквы
3.   Семплируйте следующую букву
4.   Повторяйте пункты 2-3 в цикле
5.   Преобразуйте токены в строчку

In [ ]:
model.continues_sentence("Я помню чудное мгновенье", dataset_maker.tokenizer)

In [ ]:
#continues_sentence("Мой дядя самых честных правил,", model)

In [ ]:
#continues_sentence("Четыре года потратил Деонардо на", model)

In [ ]:
#continues_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", model)